In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import itertools
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import getpass

# Start an SQL database

In [2]:


# Create a Postgresql
dbname = 'art_kickstarter4'
username = getpass.getpass('username:')
#username = raw_input(); ##change user name to match with PostgreSQL
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
# print engine.url 
if not database_exists(engine.url):
    create_database(engine.url)
print 'Database exist? '+ str(database_exists(engine.url))


username:········
Database exist? True


# Clear existing table (will be removed when deployed)

In [3]:
con = None
con = psycopg2.connect(database = dbname, user = username)

cursor = con.cursor()
command = "DROP TABLE IF EXISTS art_table;"
cursor.execute(command)
con.commit()
cursor.close()


# Scraper protytpe(s) (to be updated)

In [4]:
def scraper_prototype(text_profile,prefix):
    
    if '"name"' in text_profile:
        text_profile = re.sub('"','',text_profile)
    
    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part)>0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2,text_profile).group(1).rstrip(',') 
                for xxy1,xxy2 in zip(itemname[:-1],itemname[1:])]    
    itemnames = [x.rstrip(':') for x in itemname]

    # print itemnames
    info_dict = dict()
    for item,ans in zip(itemnames,ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict


# Loading the project info on a page using a straper prototype and save into database.

In [5]:
for pageno in range(100):

    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?state=live&category_id=1&sort=popularity&seed=2506137&page='+str(pageno)).read()
    soup = BeautifulSoup(r, 'html.parser')

    kk = soup.find_all('div', class_="js-react-proj-card")
    print('page number = '),
    print(pageno),
    print(', number of items on the page ='),
    print(len(kk))
    if len(kk)==0:
        break

    for i in range(len(kk)):
        whole_string = str(kk[i]).replace('&amp','').replace('&quot;','')

        # === Overall project info ===

        if '"name"' in whole_string:
            project_info = '"name"'+ re.findall('"name"(.+?)"creator"',whole_string)[0]
        else:
            project_info = 'name'+ re.findall('name(.+?)creator',whole_string)[0]
        project_info_dict = scraper_prototype(project_info,'project')
        df1 = pd.DataFrame.from_dict(project_info_dict,orient='index').transpose()

        # === Scrap creater string ===
        if '"creator"' in whole_string:
            creator_string = re.findall(r'\"creator\"\:\{(.+?)\}',whole_string)[0]
        else:
            creator_string = re.findall(r'creator\:\{(.+?)\}',whole_string)[0]
        creator_info_dict = scraper_prototype(creator_string,'creator')
        creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')            
        df2 = pd.DataFrame.from_dict(creator_info_dict,orient='index').transpose()

        # === Scrap location string ===
        if '"location"' in whole_string:
            location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)[0]
        else:
            location_string = re.findall(r'location\:\{(.+?)\}',whole_string)[0]
        #location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)
        location_info_dict = scraper_prototype(location_string,'location')
        df3 = pd.DataFrame.from_dict(location_info_dict,orient='index').transpose()             

        # === Scrap profile string ===
        if '"profile"' in whole_string:
            profile_string = re.findall(r'\"profile\"\:\{(.+?)\}',whole_string)[0]
        else:
            profile_string = re.findall(r'profile\:\{(.+?)\}',whole_string)[0]

        profile_info_dict = scraper_prototype(profile_string,'profile')
        df4 = pd.DataFrame.from_dict(profile_info_dict,orient='index').transpose()

        df = pd.concat((df1,df2,df3,df4),axis=1)

        # Record down columns
        if pageno == 0 and i==0:
            df_columns = df.columns

        #print 'Finish concatenating to df'
        try:            
            df.to_sql('art_table', engine, if_exists='append')
            #print 'Finish putting into art_table'            
        except:
            to_eliminate = set(df.columns.values) - set(df_columns.values)
            for to_drop in to_eliminate:
                df.drop(to_drop,axis=1,inplace=True)
            df.to_sql('art_table', engine, if_exists='append')
            #print 'Successfully handle exception'


page number =  0 , number of items on the page = 12
page number =  1 , number of items on the page = 12
page number =  2 , number of items on the page = 12
page number =  3 , number of items on the page = 12
page number =  4 , number of items on the page = 12
page number =  5 , number of items on the page = 12
page number =  6 , number of items on the page = 12
page number =  7 , number of items on the page = 12
page number =  8 , number of items on the page = 12
page number =  9 , number of items on the page = 12
page number =  10 , number of items on the page = 12
page number =  11 , number of items on the page = 12
page number =  12 , number of items on the page = 12
page number =  13 , number of items on the page = 12
page number =  14 , number of items on the page = 12
page number =  15 , number of items on the page = 12
page number =  16 , number of items on the page = 12
page number =  17 , number of items on the page = 12
page number =  18 , number of items on the page = 12
pag

# Check what's on the SQL database

In [7]:
sql_query = """
SELECT * FROM art_table;
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)

print(birth_data_from_sql.shape)

birth_data_from_sql

(296, 53)


,index,project_pledged,project_name,project_currency_symbol,project_slug,project_currency_trailing_code,project_disable_communication,project_country,project_state,project_blurb,...,profile_show_feature_image,profile_should_show_feature_image_section,profile_text_color,profile_name,profile_link_text_color,profile_background_color,profile_state_changed_at,profile_link_url,profile_state,profile_feature_image_attributes
0,0,25794.0,The Sketching Tin,£,the-sketching-tin,false,false,GB,live,A portable kit for drawing ; painting. Small e...,...,false,true,null,null,null,null,1501498018,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
1,0,21105.0,Trevor Paglen: Orbital Reflector,$,trevor-paglen-orbital-reflector,true,false,US,live,"In collaboration with Nevada Museum of Art, ar...",...,false,true,null,null,null,null,1503002429,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
2,0,38045.0,Celara: 2018 Calendar,$,celara-2018-calendar,true,false,US,live,"Hi! My name is Karen Hallion, and I am an arti...",...,false,true,null,null,null,null,1502299545,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
3,0,52767.0,Ai Weiwei: Good Fences Make Good Neighbors,$,ai-weiwei-good-fences-make-good-neighbors,true,false,US,live,Public Art Fund ; Ai Weiwei collaborate on a p...,...,false,true,null,null,null,null,1500998996,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
4,0,10503.0,WTF CARDISTRY - THE NEXT LEVEL OF FUNCTIONAL D...,$,wtf-cardistry-the-next-level-of-functional-design,true,false,US,live,WTF Cardistry - The NEXT LEVEL in Cardistry De...,...,false,true,null,null,null,null,1504543727,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
5,0,2470.0,Fantastical Coloring Book #3,$,fantastical-coloring-book-3,true,false,US,live,Over 70 fantasy drawings for you to color incl...,...,false,true,null,null,null,null,1501515594,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
6,0,458.0,Rocky Horror Picture Show Switchblade Knife En...,£,rocky-horror-picture-show-switchblade-knife-en...,false,false,GB,live,Are you a NO GOOD KID like EDDIE? I want to ma...,...,false,true,null,null,null,null,1504731726,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
7,0,640.0,ETA Tarot Deck | 80 Artist Collaborative Project,$,eta-tarot-deck-80-artist-collaborative-project,true,false,CA,live,Help support the Emerging Toronto Artists comm...,...,false,true,null,null,null,null,1501543346,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
8,0,1583.0,A Fairy Enchanting 2018 calendar by Meredith D...,$,a-fairy-enchanting-2018-calendar-by-meredith-d...,true,false,US,live,Meredith Dillman wants to make 2018 a fairy en...,...,false,true,null,null,null,null,1499968962,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
9,0,12009.0,Ink the Earth,$,ink-the-earth,true,false,US,live,A collection of circular artwork focusing on t...,...,false,true,null,null,null,null,1501874347,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...
